In [30]:
from keras.models import Sequential
from keras.layers import Conv2D, Input, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam

import numpy
import math
import cv2

import os
import h5py
import time

In [31]:
FILENAME_TRAIN = "dataTrain.h5"
FILENAME_VALIDATE = "dataValidate.h5"
FILENAME_CHECKPOINT = ""

EPOCHS = 1
BATCH_SIZE = 128

In [32]:
def srcnnTrain():
    # lrelu = LeakyReLU(alpha=0.1)
    SRCNN = Sequential()
    SRCNN.add(Conv2D(nb_filter=128, nb_row=9, nb_col=9, init='glorot_uniform',
                     activation='relu', border_mode='valid', bias=True, input_shape=(32, 32, 1)))
    SRCNN.add(Conv2D(nb_filter=64, nb_row=3, nb_col=3, init='glorot_uniform',
                     activation='relu', border_mode='same', bias=True))
    # SRCNN.add(BatchNormalization())
    SRCNN.add(Conv2D(nb_filter=1, nb_row=5, nb_col=5, init='glorot_uniform',
                     activation='linear', border_mode='valid', bias=True))
    adam = Adam(lr=0.0003)
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN

In [33]:
def h5DataRead(file):
    with h5py.File(file, 'r') as hf:
        data = numpy.array(hf.get('data'))
        label = numpy.array(hf.get('label'))
        train_data = numpy.transpose(data, (0, 2, 3, 1))
        train_label = numpy.transpose(label, (0, 2, 3, 1))
        return train_data, train_label

In [34]:
tick1 = cv2.getTickCount()

In [35]:
srcnn_model = srcnnTrain()
print(srcnn_model.summary())
data, label = h5DataRead(FILENAME_TRAIN)
val_data, val_label = h5DataRead(FILENAME_VALIDATE)

srcnn_model.fit(data, label, batch_size=BATCH_SIZE, validation_data=(val_data, val_label),
                shuffle=True, epochs=EPOCHS, verbose=1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", input_shape=(32, 32, 1..., kernel_size=(9, 9), filters=128, padding="valid", kernel_initializer="glorot_uniform", use_bias=True)`
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", kernel_size=(3, 3), filters=64, padding="same", kernel_initializer="glorot_uniform", use_bias=True)`
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="linear", kernel_size=(5, 5), filters=1, padding="valid", kernel_initializer="glorot_uniform", use_bias=True)`
  # Remove the CWD from sys.path while we load stuff.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 24, 24, 128)       10496     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 24, 24, 64)        73792     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 20, 20, 1)         1601      
Total params: 85,889
Trainable params: 85,889
Non-trainable params: 0
_________________________________________________________________
None
Train on 22000 samples, validate on 2130 samples
Epoch 1/1
22000/22000 [==============================] - 876s 40ms/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 2.3247e-05 - val_mean_squared_error: 2.3247e-05


In [36]:
score = srcnn_model.evaluate(val_data, val_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.32465926133e-05
Test accuracy: 2.32467985656e-05


In [37]:
fileName = 'yayoi_' + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.h5'
srcnn_model.save_weights(fileName)
print(fileName + ' saved')

yayoi_2018-07-06_062557.h5 saved


In [38]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())
print("processed time: " + str(tick) + " ms")

processed time: 904172 ms
